# OpenSMILE Analysis
This notebook loads OpenSMILE csv- data, cleans and plots it

## Import relevant libraries

In [1]:
import numpy as np
import pandas as pd
from os import listdir
import matplotlib.pyplot as plt
import itertools as it
from statsmodels.sandbox.stats.multicomp import multipletests
import statsmodels.api as sm
#import nltk
import scipy.stats as st
import statsmodels.formula.api as smf
import seaborn as sns
import Helper as hp

## Load .csv data with results of OpenSMILE Analysis
First we load .csv data and clean it (removing of NaNs), then we store information of all files in seperate panda dataframes containing information about affect, emotion and valence/arousal for all participants.

In [2]:
data = pd.read_csv("UIST2019_OpenSMILE.csv")

#Set Labels 
emotion_label = ['Anger', 'Boredom', 'Disgust', 'Fear', 'Happiness', 'Emo_Neutral', 'Sadness']
affect_label = ['Aggressiv', 'Cheerful', 'Intoxicated', 'Nervous', 'Aff_Neutral', 'Tired']
loi_label = ['Disinterest', 'Normal', 'High Interest']

#Get specific data and save it into new data frames
# We use the pandas .copy(deep=True) function to prevent the SettingWithCopyWarning we would otherwise get. Since we do
# not write, but only read from the data, the warning does not affect the data frames
df_emotion = data[['Anger', 'Boredom', 'Disgust', 'Fear', 'Happiness', 'Emo_Neutral', 'Sadness', 'Filename']].copy(deep=True)
df_affect = data[['Aggressiv', 'Cheerful', 'Intoxicated', 'Nervous', 'Aff_Neutral', 'Tired', 'Filename']].copy(deep=True)
df_loi = data[['Disinterest', 'Normal', 'High Interest', 'Filename']].copy(deep=True)
df_ar_val = data[['Arousal', 'Valence', 'Filename']].copy(deep=True)
#For further usage, we want to append the CharacterID as a column, which is saved with other information in the filename
#Since we only want the digits, we can remove all non-digit characters of the filename column and append the column to the df

df_emotion['Char_ID'] = df_emotion['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_affect['Char_ID'] = df_affect['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_loi['Char_ID'] = df_loi['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_ar_val['Char_ID'] = df_ar_val['Filename'].replace('\D+','', regex = True).copy(deep=True)

## Let's load information about the speakers
The speaker ID is saved in a single .csv file containing four important columns: ID, Age, Sex and Acadedmic Status. Since before loaded OpenSMILE csv files are named using the corresponding index (ex. speaker with id 0 has two files 0_a.csv and 0_b.csv), so that a link can be created

In [3]:
char_data = pd.read_csv("UIST2019_CharacterData.csv")  

#Join above tables and Character Tables

#To Join DataFrames we have to cast the column on which we want to join to int, so that both columns have the same data type
char_data['ID'] = char_data['ID'].astype(int)
df_ar_val['Char_ID'] = df_ar_val['Char_ID'].astype(int)
df_emotion['Char_ID'] = df_emotion['Char_ID'].astype(int)
df_affect['Char_ID'] = df_affect['Char_ID'].astype(int)
df_loi['Char_ID'] = df_loi['Char_ID'].astype(int)

#Safe new data frames
df_ar_val_char = df_ar_val.merge(char_data, how = 'left', left_on='Char_ID', right_on='ID')
df_emotion_char = df_emotion.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')
df_affect_char = df_affect.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')
df_loi_char = df_loi.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')

## Chi-squared Test of Independence
We Start with characteristic sex. The null hypothesis states that the two categorical variables sex and e.g. emotion are independent.

For that we convert the previously used data frames which contain probabilities of e.g. emotion, affect etc. to frequency tables using the helper method calcFrequencyTable(). For each row (sample) the function selects the maximum probability and counts it as an occured frequency. Furthermore the function takes in an integer so that the matching labels for emotion are generated (voice features). The second integer defines the character feature e.g. Age, so that it can split the data set into a form of n x m, where n stands for the number of character features (in most cases it's 2, but for age it's 3) and m stands for the number of voice features e.g. number of different emotions.  
The frequency tables are then used as input for the stats.chi2_contingency() function.

If the p-value is significant, residuals are computed to identify the cells, in which the groups differ, i.e. the data sets differ in the emotion 'anger', which contributes to the significant p-value.

In [8]:
emo_sex_chi2 = hp.chi2(df_emotion_char, 'Sex', 0, True)
aff_sec_chi2 = hp.chi2(df_affect_char, 'Sex', 1, True)
ar_val_sec_chi2 = hp.chi2(df_ar_val_char, 'Sex', 2, True)
loi_sec_chi2 = hp.chi2(df_loi_char, 'Sex', 3, True)

TypeError: reduction operation 'argmax' not allowed for this dtype

Now move on to academic status, the hypothesis being that the variables academic status and e.g. emotion are independent.

In [ ]:
emo_aca_chi2 = hp.chi2(df_emotion_char, 'Academic Status', 0, True)
aff_aca_chi2 = hp.chi2(df_affect_char, 'Academic Status', 1, True)
ar_val_aca_chi2 = hp.chi2(df_ar_val_char, 'Academic Status', 2, True)
loi_aca_chi2 = hp.chi2(df_loi_char, 'Academic Status', 3, True)

Now let's look if age and e.g. emotion/ affect/ arousal-valence/ level of interest are independent

In [ ]:
emo_age_chi2 = hp.chi2(df_emotion_char, 'Age', 0, True)
aff_age_chi2 = hp.chi2(df_affect_char, 'Age', 1, True)
ar_val_age_chi2 = hp.chi2(df_ar_val_char, 'Age', 2, True)
loi_age_chi2 = hp.chi2(df_loi_char, 'Age', 3, True)

## Post-Hoc tests for age, as it has three different groups

In [ ]:
print('post-hoc emotions and different groups')
emo_reject_list, emo_corrected_p_vals, emo_combinations, emo_residuals= hp.chi2_post_hoc(emo_age_chi2[1], 'bonferroni', True, True)
print('\n post-hoc affect and different groups')
aff_reject_list, emo_corrected_p_vals, emo_combinations, aff_residuals = hp.chi2_post_hoc(aff_age_chi2[1], 'bonferroni', True, True)
print('\n post-hoc arousal-valence and different groups')
ar_val_reject_list, ar_val_corrected_p_vals, ar_val_combinations, ar_val_residuals = hp.chi2_post_hoc(ar_val_age_chi2[1], 'bonferroni',True, True)
print('\n post-hoc level of intereset and different groups')
loi_reject_list, loi_corrected_p_vals, loi_combinations, loi_residuals = hp.chi2_post_hoc(loi_age_chi2[1], 'bonferroni', True, True)

# Test Chi2

In [ ]:
# Get Emotion dataset and drop all character columns but gender
df_test = df_emotion_char.drop(['CharacterID', 'file', 'Age', 'Academic Status'], axis = 1)

test_sadness_m = df_test.loc[df_test['Sex'] == 0.0]['sadness']
test_sadness_f = df_test.loc[df_test['Sex'] == 1.0]['sadness']
test_m, counts_m = np.unique(np.around(test_sadness_m, 1), return_counts = True)
test_f, counts_f = np.unique(np.around(test_sadness_f, 1), return_counts = True)
data = ['Sex', '0.0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1.0']
cont_table = pd.DataFrame(columns = data)
test_m = np.around(np.arange(0, 1.1, 0.1),1)

for i in range(0,11):
    print()
    #print(test_m[i])



#row_m = ['male']
#cont_table = cont_table['Sex'].append(row_m, ignore_index = True)

In [ ]:
test_m

In [ ]:
df_test